# Clase 14: Optimización modelo de inventarios (en Julia)

 Prof. Tito Homem-de-Mello

In [ ]:
#import Pkg; Pkg.add("JuMP");
#import Pkg; Pkg.add("HiGHS"); 
#import Pkg; Pkg.add("Statistics");
#import Pkg; Pkg.add("Random");
#import Pkg; Pkg.add("Distributions");

In [ ]:
using JuMP
using HiGHS
using Statistics
using Random
using Distributions

### Parámetros del modelo:

In [ ]:
r = 300   #Precio de compra
v =  1500  #Precio de venta
h = r     #Costo inventario
q = v     #Costo quiebre de inventario

## Veamos el caso general con *N muestras*. 


### Queremos resolver
### $\max_{s\geq 0}⁡ \frac{1}{N} \sum_{n=1}^N U(s,D_n)$

### donde
### $U(s,D)=−[h∗\max (s-D,0)+q∗\max (D-s,0)]+ (v-r)∗D$

### Primero generamos $N=100$ muestras de la demanda:

In [ ]:
N = 100
Distr = DiscreteUniform(100,400)
D = rand(Distr,N);

### El modelo de optimización lineal:

In [ ]:
model = Model(HiGHS.Optimizer)

In [ ]:
@variable(model, zo[i=1:N] >= 0);
@variable(model, zu[i=1:N] >= 0);
@variable(model, s >= 0); 

In [ ]:
@constraint(model, [i in 1:N], zo[i] - s >= -D[i]);
@constraint(model, [i in 1:N], zu[i] + s >= D[i]); 

In [ ]:
@objective(model, Min, (1/N)*sum(h*zo[i] + q*zu[i] for i in 1:N));

In [ ]:
optimize!(model)

In [ ]:
println("Solución óptima = ",value(s))
println("Valor óptimo = ",(v-r)*mean(D)-objective_value(model))

## Ahora veamos el caso con 2 productos, con capacidad máxima de la bodega

In [ ]:
r1 = 300   #Precio de compra prod. 1
v1 = 1500  #Precio de venta prod. 1
h1 = r1     #Costo inventario prod. 1
q1 = v1     #Costo quiebre de inventario prod. 1

r2 = 1000   #Precio de compra prod. 2
v2 = 3000  #Precio de venta prod. 2
h2 = r2     #Costo inventario prod. 2
q2 = v2;     #Costo quiebre de inventario prod. 2

In [ ]:
N = 10000
Distr1 = DiscreteUniform(100,400)
Dp1 = rand(Distr1,N);
Distr2 = DiscreteUniform(50,150)
Dp2 = rand(Distr2,N);
Cap = 300   #Capacidad de la bodega

model = Model(HiGHS.Optimizer)
@variable(model, zo1[i=1:N] >= 0);
@variable(model, zu1[i=1:N] >= 0);
@variable(model, s1 >= 0); 
@variable(model, zo2[i=1:N] >= 0);
@variable(model, zu2[i=1:N] >= 0);
@variable(model, s2 >= 0);  
@constraint(model, [i in 1:N], zo1[i] - s1 >= -Dp1[i]) 
@constraint(model, [i in 1:N], zu1[i] + s1 >= Dp1[i]) 
@constraint(model, [i in 1:N], zo2[i] - s2 >= -Dp2[i]) 
@constraint(model, [i in 1:N], zu2[i] + s2 >= Dp2[i]) 
@constraint(model, s1+s2 <= Cap) 
@objective(model, Min, (1/N)*sum(h1*zo1[i] + q1*zu1[i] + h2*zo2[i] + q2*zu2[i] for i in 1:N));
optimize!(model)

In [ ]:
println("Solución óptima producto 1 = ",value(s1))
println("Solución óptima producto 2 = ",value(s2))
println("Valor óptimo = ",(v1-r1)*mean(Dp1)+(v2-r2)*mean(Dp2)-objective_value(model))
